In [1]:
import numpy as np
import tensorflow.keras.backend as k
from tensorflow.keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import cv2
import datetime

In [2]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,

        )

model.save('mymodel.h5',model_saved)

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


D:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
83/83 [==============================] - 47s 547ms/step - loss: 0.4915 - accuracy: 0.7565 - val_loss: 0.1767 - val_accuracy: 0.9227
Epoch 2/10
83/83 [==============================] - 41s 491ms/step - loss: 0.1800 - accuracy: 0.9292 - val_loss: 0.1295 - val_accuracy: 0.9536
Epoch 3/10
83/83 [==============================] - 41s 499ms/step - loss: 0.1227 - accuracy: 0.9589 - val_loss: 0.0885 - val_accuracy: 0.9691
Epoch 4/10
83/83 [==============================] - 42s 503ms/step - loss: 0.1123 - accuracy: 0.9597 - val_loss: 0.1029 - val_accuracy: 0.9588
Epoch 5/10
83/83 [==============================] - 42s 496ms/step - loss: 0.0985 - accuracy: 0.9589 - val_loss: 0.1084 - val_accuracy: 0.9742
Epoch 6/10
83/83 [==============================] - 42s 500ms/step - loss: 0.1076 - accuracy: 0.9612 - val_loss: 0.1124 - val_accuracy: 0.9742
Epoch 7/10
83/83 [==============================] - 44s 526ms/step - loss: 0.0793 - accuracy: 0.9703 - val_loss: 0.1334 - val_accuracy: 0.9433

In [4]:
mymodel=load_model('mymodel.h5')
test_image=image.load_img(r'D:/FaceMaskDetector/test/with_mask/5-with-mask.jpg',target_size=(150,150,3))
test_image
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
mymodel.predict(test_image)[0][0]

0.0

In [7]:
test_image=image.load_img(r'D:/FaceMaskDetector/test/without_mask/0.jpg',target_size=(150,150,3))
test_image
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
mymodel.predict(test_image)[0][0]

1.0

In [8]:
mymodel=load_model('mymodel.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()